In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=8455a108d7d11d4fff28a73c71692c6b4ab04cb3723a811839802be9c5df6693
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import col
import numpy as np
import os

In [3]:
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"
os.environ["HADOOP_HOME"] = "C:/winutils"

# Creating spark session
spark = SparkSession.builder.appName("Crime Analytics").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

# Loading the dataset
KCDPFinal = spark.read.format("csv").option("header", True).option("inferSchema", True).option("delimiter", ",").load("/content/drive/MyDrive/DBMS_project_final/Source Code/1-Datasets/KCPD_Crime_Data/KCcrimeForAnalytics.csv").withColumnRenamed("Firearm_Used_Flag", "label")
KCDPFinal

# Create vector assembler for feature columns
VAssembler = VectorAssembler(inputCols=KCDPFinal.columns[1:19], outputCol="features")
KCDPFinal = VAssembler.transform(KCDPFinal)

# Split the crime dataset into training and testing data sets
trainingData, testingData = KCDPFinal.select("label", "features").randomSplit([0.7, 0.3])

# Using the training set for the model traning
from pyspark.ml.classification import NaiveBayes
NaiveBayesModel = NaiveBayes()
model = NaiveBayesModel.fit(trainingData)

# Generate prediction from test dataset
CrimepredKC = model.transform(testingData)

# Evuluate the accuracy of the model
evaluator = MulticlassClassificationEvaluator()
accuracy = evaluator.evaluate(CrimepredKC)

# Show model accuracy
print("Accuracy:", accuracy)

Accuracy: 0.624054863389264


In [4]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/output_folder/Reported_year=2010/part-00000-2b98d6cc-f2c0-4c81-b6af-d1e0f599ca21.c000.csv")
data.drop(["Unnamed: 24","N"],axis=1,inplace=True)
data.columns=[['Report_No','Reported_month', 'Reported_day',
       'Reported_hour', 'Reported_minute', 'From_year', 'From_month',
       'From_day', 'From_hour', 'From_minute', 'Offense', 'IBRS',
       'Description', 'Beat', 'Address', 'City', 'Zip_Code', 'Rep_Dist',
       'Area', 'DVFlag', 'Invl_No', 'Involvement', 'Race', 'Sex']]
data.drop("Report_No",axis=1,inplace=True)
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
data['Description']=lb.fit_transform(data['Description'])
data['Address']=lb.fit_transform(data['Address'])
data['City']=lb.fit_transform(data['City'])
data['Rep_Dist']=lb.fit_transform(data['Rep_Dist'])
data['Area']=lb.fit_transform(data['Area'])
data['DVFlag']=lb.fit_transform(data['DVFlag'])
data['Involvement']=lb.fit_transform(data['Involvement'])
data['Race']=lb.fit_transform(data['Race'])
data['Sex']=lb.fit_transform(data['Sex'])
X = data.loc[:, data.columns != "Area"]
y = data["Area"]

<ipython-input-4-2f666f6a92d1>:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  data.drop("Report_No",axis=1,inplace=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/li

In [5]:
count = data.isna().sum()
print(count)

Reported_month     0
Reported_day       0
Reported_hour      0
Reported_minute    0
From_year          0
From_month         0
From_day           0
From_hour          0
From_minute        0
Offense            0
IBRS               0
Description        0
Beat               0
Address            0
City               0
Zip_Code           0
Rep_Dist           0
Area               0
DVFlag             0
Invl_No            0
Involvement        0
Race               0
Sex                0
dtype: int64


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
from sklearn.naive_bayes import GaussianNB
GaussianModel = GaussianNB()
GaussianModel.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [8]:
y_pred= GaussianModel.predict(X_test)

In [9]:
y_pred

array([0, 2, 1, 1, 1, 1, 2, 5, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 0, 3, 2, 0,
       1, 0, 1, 3, 1, 2, 1, 1, 1, 3, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0, 5, 1,
       2, 1, 2, 2, 1, 1, 0, 2, 0, 0, 2, 0, 3, 1, 1, 1, 2, 2, 5, 1, 0, 1])

In [10]:
#testing_input()
#Area_prediction using sample input
output=GaussianModel.predict([[4.0, 15.0, 14.0, 43.0, 2010.0, 4.0, 15.0, 14.0, 43.0, 1352.0, 280.0, 0.0, 315.0, 66.0, 0.0, 64123.0, 17.0, 1.0, 1.0, 1.0, 2.0, 2.0,2.0]
])
print(output)

[1]
